<a href="https://colab.research.google.com/github/hungbuile04/Generic-Algorithm-in-solving-equation/blob/main/Generic_Algorithm_in_solving_equation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font color="blue">**PROJECT 1**</font>
##<font color="green"> **Topic:** Developing a Graphical Interface Program to Simulate the Application of Genetic Algorithms in Solving Equations.
##<font color="navy"> //Bùi Lê Hùng 20224994


**Declaring the libraries used**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import math
import time
import sympy as sp


In [ ]:
!pip install gradio
import gradio as gr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


**Environment variables**

In [ ]:
POPULATION_SIZE = 100
MUTATION_RATE = 0.03
CROSSOVER_RATE = 0.8
GENERATIONS = 60

**Init population**

In [ ]:
def generate_population(lowerBound, upperBound):
    population = np.random.uniform(lowerBound, upperBound, POPULATION_SIZE)
    return population.tolist()
print(generate_population(-10,10))

[9.834676784405818, 4.664736641182966, 2.561862094891165, 9.70324416507501, 1.7379848388825394, 3.4091473703620263, -9.387412125167911, -3.0037437155221003, 0.36685366276472386, 9.579248336843975, 5.769577282469314, -1.2297110829633908, -9.002085885029782, 3.1096913490733726, -9.106001309197541, 0.48521567491589757, -5.130988095050684, -7.717989586971756, -0.9950098169441084, -0.8535871672873636, 2.1573929563352046, 5.084354576378775, -9.417486470083755, -4.260624019347892, -5.498021917642443, -1.9939838295560168, -8.576813135929385, -0.8604679869668814, 2.2887924360396923, 7.241602468727201, 3.2519410332669434, 3.5900947221885797, 8.65941072940263, -8.310460049663371, -1.188773161072021, 8.012162645777277, 3.3636716321059783, -9.692661963041978, 3.2322537204431576, 8.442027919552942, 3.023173746501337, -2.0289247942466364, -0.5447678438453458, 4.122273107752637, -1.2347139229154358, -9.618000971424511, 9.955851493231972, 6.842852076538929, -1.160142840839855, -2.3725287156538055, -3.2

**Calculate fitness**

In [ ]:
import sympy as sp

def calculate_fitness(equation, value):
    """
    Parse the input equation using SymPy and calculate the fitness.

    Args:
        equation (str): The equation in string format (e.g., "x^2 + 3x - 5 = 0").
        value (float): The value to substitute for the variable x.

    Returns:
        float: The fitness value (lower is better).
    """
    try:
        # Create the symbol x
        x = sp.Symbol('x')

        # Replace "^" with "**" to correctly interpret exponents
        equation = equation.replace("^", "**")

        if "=" in equation:
            # Split the equation into left and right sides
            left_side, right_side = equation.split("=", 1)

            # Convert both sides to SymPy expressions
            left_expr = sp.sympify(left_side)
            right_expr = sp.sympify(right_side)

            # Calculate fitness as the absolute difference between the two sides
            fitness = abs((left_expr - right_expr).subs(x, value))
        else:
            # Treat the input as a single expression if there's no "="
            expression = sp.sympify(equation)
            fitness = abs(expression.subs(x, value))

    except (sp.SympifyError, TypeError, ValueError) as e:
        print(f"Error while parsing or evaluating the equation: {e}")
        # Assign a high fitness value in case of errors
        fitness = float('inf')

    return fitness

# Test
equation = "x-1=2"
value = -7.114390306562845
fitness = calculate_fitness(equation, value)
print(f"Fitness: {fitness}")


Fitness: 10.1143903065628


**Crossover**

In [ ]:
def generate_random_coef():
  """Generates a random float between 0 and 1 (exclusive) using NumPy."""
  while True:
    random_number = np.random.rand()
    if random_number != 0 and random_number != 1:
      return random_number

def select_parents(population, fitness):
    # Invert fitness values to prioritize values closer to 0
    inverted_fitness = [1 / (f + 1e-10) for f in fitness]  # Add a small epsilon to avoid division by zero
    total_inverted_fitness = sum(inverted_fitness)

    # Perform roulette wheel selection based on inverted fitness
    pick = np.random.uniform(0, total_inverted_fitness)
    current = 0
    for i in range(len(population)):
        current += inverted_fitness[i]
        if current > pick:
            return population[i]

def crossover(parent1, parent2):
    if random.random() < CROSSOVER_RATE:
        crossover_point = generate_random_coef()
        if(parent1 > parent2): parent1, parent2 = parent2, parent1
        child1 = parent1*crossover_point + parent2*(1-crossover_point)
        child2 = parent1*(1-crossover_point) + parent2*crossover_point
        return child1, child2
    else:
        child1 = parent1
        child2 = parent2
    return child1, child2

#Test
parent1 = 1
parent2 = 2
child1, child2 = crossover(parent1, parent2)
print(child1, child2)


1.3768046275065897 1.6231953724934103


**Mutation**

In [ ]:
def mutate(genome):
    if random.random() < MUTATION_RATE:
        mutation = np.random.normal(0, 0.01)  # Gaussian Mutation with standard deviation = 0.01
        if random.choice([True, False]):  # Randomly choose to increase or decrease
          return genome + mutation
        else:
          return genome - mutation
    return genome

**Genetic algorithm**

In [ ]:
from concurrent.futures import ThreadPoolExecutor

def genetic_algorithm_with_plot(equation, lower_bound, upper_bound):
    # Generate the initial population
    population = generate_population(lower_bound, upper_bound)
    all_populations = [population]
    best_individual = None
    logs = []
    # Use ThreadPoolExecutor for parallel fitness computation
    with ThreadPoolExecutor() as executor:
        for generation in range(GENERATIONS):
            # Compute fitness for each individual in parallel
            fitness = list(executor.map(lambda individual: calculate_fitness(equation, individual), population))

            new_population = []
            for i in range(POPULATION_SIZE // 2):
                # Select parents based on fitness
                parent1 = select_parents(population, fitness)
                parent2 = select_parents(population, fitness)

                # Perform crossover and mutation
                child1, child2 = crossover(parent1, parent2)
                child1 = mutate(child1)
                child2 = mutate(child2)

                # Add children to the new population
                new_population.extend([child1, child2])

            # Update the population
            population = new_population
            all_populations.append(population)

            # Track the best individual in the current generation
            best_individual = min(population, key=lambda individual: calculate_fitness(equation, individual))
            log_entry = f"Generation {generation}: Best individual = {best_individual}, best fitness = {calculate_fitness(equation, best_individual)}"
            logs.append(log_entry)  # Save log for the current generation

    # Determine the best individual from the final population
    best_individual = min(population, key=lambda individual: calculate_fitness(equation, individual))

    # Plot the population evolution
    fig = plot_populations(all_populations)
    return best_individual, fig, "\n".join(logs)




**Representing the population of each generation using Matplotlib.**

In [ ]:
def plot_populations(all_populations):
    fig, ax = plt.subplots()
    for i, population in enumerate(all_populations):
        ax.scatter([i] * len(population), population, s=5)  # Draw individual in each generation
    ax.set_xlabel("Thế hệ")
    ax.set_ylabel("Giá trị cá thể")
    ax.set_title("Biểu diễn các cá thể qua các thế hệ")
    return fig

**Launching the interface using Gradio.**

In [ ]:

# Gradio interface
with gr.Blocks() as demo:
    with gr.Row():
        equation = gr.Textbox(label="Nhập phương trình", value="x^2-1=0")
        lower_bound = gr.Number(label="Giới hạn dưới", value=-1)
        upper_bound = gr.Number(label="Giới hạn trên", value=1)

    output_text = gr.Textbox(label="Cá thể tốt nhất")
    output_plot = gr.Plot(label="Biểu đồ các cá thể qua các thế hệ")
    output_logs = gr.Textbox(label="Nhật ký thế hệ", lines=10, interactive=False)

    btn = gr.Button("Chạy thuật toán")
    btn.click(
        fn=genetic_algorithm_with_plot,
        inputs=[equation, lower_bound, upper_bound],
        outputs=[output_text, output_plot, output_logs]
    )

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://51b744f2d354fc9589.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
